### Mopsy and Mocha ####

#### Introduction ###
Here in this machine learning project, we will be using the concept of convolutional linear regression, where we will be interaction a set of images, some with a dog and others with a cat. The data set has already been divided into a testing and training set, where we will begin to train a model to identify cats, dogs, and the differences between them.


Below is some code that allows for the importing of all images as a Numpy array, for which the skimage library was used. First, all related libraries were imported.

In [ ]:
import sys, os
import skimage.io, skimage.color

import pandas as pd
import numpy as np

This function returns the shape of an image, given a folder path and a filename. This is returned as a Numpy array.

In [17]:
def get_image_resolution(folder, file):
    full_path = os.path.join(folder, file)
    img_data = skimage.io.imread(full_path)
    bw_data = skimage.color.rgb2gray(img_data)
    return bw_data.shape

As a proof of concept to show that pictures could be read, all training dog pictures were opened into a Numpy array, where the `get_image_resolution()` function extracted the shape as long as the image is opened. Only one folder is needed for this proof of concept.

In [19]:
# First, we need to get every single file.
training_dog_folder = "training_set/dogs"

training_dog_paths = os.listdir(training_dog_folder)
training_dog_resolutions = [get_image_resolution(training_dog_folder, i) for i in training_dog_paths]
x = skimage.io.imread('All_Might_Hero_Form_Full_Body.png')
print(training_dog_resolutions)

[(499, 370), (499, 286), (160, 237), (215, 166), (500, 269), (500, 409), (304, 495), (240, 319), (233, 350), (254, 385), (375, 499), (359, 480), (499, 267), (374, 500), (500, 415), (375, 499), (374, 500), (280, 324), (358, 419), (375, 499), (497, 500), (399, 336), (374, 500), (379, 500), (431, 336), (500, 332), (269, 360), (500, 272), (325, 424), (500, 433), (340, 457), (500, 496), (171, 299), (122, 160), (375, 499), (375, 499), (500, 298), (418, 500), (300, 226), (447, 350), (261, 350), (374, 500), (398, 500), (395, 375), (374, 500), (485, 480), (374, 500), (374, 500), (500, 491), (422, 449), (374, 500), (288, 359), (332, 500), (418, 411), (239, 320), (375, 499), (251, 250), (392, 499), (375, 499), (187, 138), (380, 500), (500, 335), (418, 499), (500, 426), (374, 500), (387, 480), (309, 296), (251, 350), (374, 500), (210, 359), (361, 387), (479, 455), (499, 375), (337, 499), (276, 349), (154, 300), (499, 420), (265, 500), (269, 360), (499, 397), (458, 499), (500, 352), (374, 500), (32

With these resolutions, next week, the resolutions will be filtered using a histogram, so comparisons can only be made between similarly sized pictures. Since reading these resolutions requires getting the shape of each Numpy array of pixels, it indicates that every image file in a folder of pictures was opened.

Our progress is available at https://github.com/andy64734/all-might.